# Import Libraries

In [2]:
import pandas as pd
import numpy as np 
from numpy import nan
import tweepy
import requests
import json
import os
import re
import matplotlib.pyplot as plt
import datetime
import sys

# Gather

In [3]:
archive = pd.read_csv("./twitter-archive-enhanced.csv")

In [4]:
ImageURL = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"

response = requests.get(ImageURL)

with open(os.path.join(os.getcwd(), ImageURL.split('/')[-1]), mode = 'wb') as file:
    file.write(response.content)
    
imagePrediction = pd.read_csv('./image-predictions.tsv',sep='\t')

In [5]:
# Personal API keys, secrets, and tokens have been replaced with placeholders
consumer_key = 'xvWlbHKWXDHUYwFJiW4EpKQoe'
consumer_secret ='M7xWLk2p2lemGtlsksz1NFau9MfRZXbQJkEVratJ5NMUlxeIrD'
access_token = '182197942-XPars3NGLO1BI9BbMtJqoa65piK2atTmTa62LB87'
access_secret = 'MiXDOwIf5yOLNP1deDX72ew9JoqL69P1Z92HV5gmoEtFk'


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth_handler=auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [6]:
id_errors = []
def get_all_tweets(screen_name):       
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)
    alltweets = []
    alltweets.extend(new_tweets)

    oldest = alltweets[-1].id - 1

    while len(new_tweets) > 0:
        try:
            new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
    
            alltweets.extend(new_tweets)

            oldest = alltweets[-1].id - 1

            print("%s completed" % (len(alltweets)))
            
        except Exception as e:          
            print(new_tweets.id_str + ": " + str(e))
            id_errors.append(new_tweets)
            continue

    outtweets = [[tweet.id_str, tweet.retweet_count,tweet.favorite_count]  for tweet in alltweets] 
    df = pd.DataFrame.from_dict(outtweets)
        
    df.to_csv('./tweet_json.txt', header =True,index=None, sep=' ', mode='w')
    print('True')

if __name__ == '__main__':
    get_all_tweets("dog_rates")

399 completed
599 completed
799 completed
999 completed
1199 completed
1399 completed
1599 completed
1799 completed
1999 completed
2157 completed
2157 completed
True


In [7]:
tweet_json = pd.read_csv('./tweet_json.txt',sep=' ')
tweet_json.columns =["tweet_id","retweet_count","favorite_count"]

# Access 

In [8]:
archive

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


archive table  has 2356 rows and 17 columns . 
1. These dog stage columns are too redundent (doggo, floofer, pupper, and puppo, ...) which result into a tidyness issue.  -->  merge them into one dog_stage column. 
2. in_reply_to_status_id,in_reply_to_user_idretweeted_status_id,retweeted_status_user_id and retweeted_status_timestamp have a lot of missing value --> examine more in programmatical portion.
3. Timestamp varaible has the suffix '+0000".
4. Source column's value has "<a href=" and ".r" around text. some rows have the same source url.
5. Some naame variable has 'a', 'an' and 'the'.  
6. The value inside 'text' is incomplete.

In [9]:
imagePrediction

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


imagePrediction table has 2075 rows and 12 columns. 
1. In p1, p2, p3 columns (breeds of dogs) 
2. Three confidence have 6 decimals.
3. 'Newfoundland','Lhasa' in p2 and p1 are not a breed of dog, but indicator = TRUE

In [10]:
tweet_json

,tweet_id,retweet_count,favorite_count
0,1090301319825436672,2,305
1,1090300780270149632,11,1226
2,1090300624657252352,6989,79691
3,1090060665903927296,1,143
4,1090060504788197377,1,116
5,1090058668987404289,58,1282
6,1090055878013644800,3,1352
7,1090054426788691968,10442,120078
8,1090051186432958464,0,11
9,1090006105990750209,0,102


tweet_json table has 3225 rows and 5 columns.

#### 2. Programatical assessment 

#### 2.1 archive table

In [11]:
archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

1. timestamp variable's type should be time format rather than string. 
2. Expanded_urls,in_reply_to_status_id,in_reply_to_user_id only has 78 non-null value. 
3. retweeted_status_id,retweeted_status_user_id and retweeted_status_timestamp have 2175 missing value 
4. Tweet_ID should be string rather than numeric value 

In [12]:
archive.tail()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
2351,666049248165822465,NaN,NaN,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a 1949 1st generation vulpix. Enj...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666049248...,5,10,None,None,None,None,None
2352,666044226329800704,NaN,NaN,2015-11-16 00:04:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a purebred Piers Morgan. Loves to Netf...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666044226...,6,10,a,None,None,None,None
2353,666033412701032449,NaN,NaN,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a very happy pup. Big fan of well-main...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666033412...,9,10,a,None,None,None,None
2354,666029285002620928,NaN,NaN,2015-11-15 23:05:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a western brown Mitsubishi terrier. Up...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666029285...,7,10,a,None,None,None,None
2355,666020888022790149,NaN,NaN,2015-11-15 22:32:08 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a Japanese Irish Setter. Lost eye...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666020888...,8,10,None,None,None,None,None


In [13]:
pd.set_option('display.max_colwidth', -1)

In [14]:
archive['name'].value_counts()

None          745
a             55 
Charlie       12 
Lucy          11 
Oliver        11 
Cooper        11 
Tucker        10 
Penny         10 
Lola          10 
Bo            9  
Winston       9  
the           8  
Sadie         8  
Toby          7  
Bailey        7  
an            7  
Buddy         7  
Daisy         7  
Bella         6  
Leo           6  
Koda          6  
Jax           6  
Scout         6  
Oscar         6  
Jack          6  
Milo          6  
Rusty         6  
Dave          6  
Stanley       6  
Bentley       5  
             ..  
Karl          1  
Katie         1  
Jiminus       1  
Chesterson    1  
Goose         1  
Jessifer      1  
Tobi          1  
Rueben        1  
Sojourner     1  
Karma         1  
Asher         1  
Rooney        1  
Shelby        1  
Wishes        1  
Rizzo         1  
all           1  
Spanky        1  
Kawhi         1  
Willie        1  
Pepper        1  
Brian         1  
Carter        1  
Sephie        1  
Chef          1  
Grizzwald 

The name variable has 745 'None', 55 'a',  7 'an' name and 8 'the'. 

In [15]:
archive.rating_denominator.value_counts()

10     2333
11     3   
50     3   
80     2   
20     2   
2      1   
16     1   
40     1   
70     1   
15     1   
90     1   
110    1   
120    1   
130    1   
150    1   
170    1   
7      1   
0      1   
Name: rating_denominator, dtype: int64

There is 1 record with denominator = 0, which is invalid rating.

In [16]:
archive[archive['rating_denominator'] ==0]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
313,835246439529840640,8.352460e+17,26259576.0,2017-02-24 21:54:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","@jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho",NaN,NaN,NaN,NaN,960,0,None,None,None,None,None


In [17]:
archive[archive['rating_denominator'] ==2]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
2335,666287406224695296,NaN,NaN,2015-11-16 16:11:11 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is an Albanian 3 1/2 legged Episcopalian. Loves well-polished hardwood flooring. Penis on the collar. 9/10 https://t.co/d9NcXFKwLv,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666287406224695296/photo/1,1,2,an,None,None,None,None


In [18]:
archive[archive['rating_denominator'] ==7] 


,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
516,810984652412424192,NaN,NaN,2016-12-19 23:06:23 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. \nKeep Sam smiling by clicking and sharing this link:\nhttps://t.co/98tB8y7y7t https://t.co/LouL5vdvxx,NaN,NaN,NaN,"https://www.gofundme.com/sams-smile,https://twitter.com/dog_rates/status/810984652412424192/photo/1",24,7,Sam,None,None,None,None


In [19]:
archive[archive['rating_denominator'] ==11] 

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
784,775096608509886464,NaN,NaN,2016-09-11 22:20:06 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","RT @dog_rates: After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https:/…",7.403732e+17,4.196984e+09,2016-06-08 02:41:38 +0000,"https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1",9,11,None,None,None,None,None
1068,740373189193256964,NaN,NaN,2016-06-08 02:41:38 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https://t.co/XAVDNDaVgQ",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1",9,11,None,None,None,None,None
1662,682962037429899265,NaN,NaN,2016-01-01 16:30:13 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Darrel. He just robbed a 7/11 and is in a high speed police chase. Was just spotted by the helicopter 10/10 https://t.co/7EsP8LmSp5,NaN,NaN,NaN,https://twitter.com/dog_rates/status/682962037429899265/photo/1,7,11,Darrel,None,None,None,None


In [20]:
archive.rating_numerator.value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7       55 
14      54 
5       37 
6       32 
3       19 
4       17 
1       9  
2       9  
420     2  
0       2  
15      2  
75      2  
80      1  
20      1  
24      1  
26      1  
44      1  
50      1  
60      1  
165     1  
84      1  
88      1  
144     1  
182     1  
143     1  
666     1  
960     1  
1776    1  
17      1  
27      1  
45      1  
99      1  
121     1  
204     1  
Name: rating_numerator, dtype: int64

In [21]:
archive[archive['rating_numerator'] == 1776]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
979,749981277374128128,NaN,NaN,2016-07-04 15:00:45 +0000,"<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",This is Atticus. He's quite simply America af. 1776/10 https://t.co/GRXwMxLBkh,NaN,NaN,NaN,https://twitter.com/dog_rates/status/749981277374128128/photo/1,1776,10,Atticus,None,None,None,None


In [22]:
archive[archive['rating_numerator'] ==0]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
315,835152434251116546,NaN,NaN,2017-02-24 15:40:31 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",When you're so blinded by your systematic plagiarism that you forget what day it is. 0/10 https://t.co/YbEJPkg4Ag,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/835152434251116546/photo/1,https://twitter.com/dog_rates/status/835152434251116546/photo/1,https://twitter.com/dog_rates/status/835152434251116546/photo/1",0,10,None,None,None,None,None
1016,746906459439529985,7.468859e+17,4.196984e+09,2016-06-26 03:22:31 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","PUPDATE: can't see any. Even if I could, I couldn't reach them to pet. 0/10 much disappointment https://t.co/c7WXaB2nqX",NaN,NaN,NaN,https://twitter.com/dog_rates/status/746906459439529985/photo/1,0,10,None,None,None,None,None


In [23]:
archive[archive['rating_numerator'] == 5]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
45,883482846933004288,NaN,NaN,2017-07-08 00:28:19 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/883482846933004288/photo/1,https://twitter.com/dog_rates/status/883482846933004288/photo/1",5,10,Bella,None,None,None,None
730,781661882474196992,NaN,NaN,2016-09-30 01:08:10 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Who keeps sending in pictures without dogs in them? This needs to stop. 5/10 for the mediocre road https://t.co/ELqelxWMrC,NaN,NaN,NaN,https://twitter.com/dog_rates/status/781661882474196992/photo/1,5,10,None,None,None,None,None
956,751583847268179968,NaN,NaN,2016-07-09 01:08:47 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Please stop sending it pictures that don't even have a doggo or pupper in them. Churlish af. 5/10 neat couch tho https://t.co/u2c9c7qSg8,NaN,NaN,NaN,https://twitter.com/dog_rates/status/751583847268179968/photo/1,5,10,None,doggo,None,pupper,None
1399,699691744225525762,NaN,NaN,2016-02-16 20:28:06 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Dave. He's a tropical pup. Short lil legs (dachshund mix?) Excels underwater, but refuses to eat kibble 5/10 https://t.co/ZJnCxlIf62",NaN,NaN,NaN,https://twitter.com/dog_rates/status/699691744225525762/photo/1,5,10,Dave,None,None,None,None
1461,694925794720792577,NaN,NaN,2016-02-03 16:49:55 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine - Make a Scene</a>",Please only send in dogs. This t-rex is very scary. 5/10 ...might still pet (vid by @helizabethmicha) https://t.co/Vn6w5w8TO2,NaN,NaN,NaN,https://vine.co/v/iJvUqWQ166L,5,10,None,None,None,None,None
1508,691483041324204033,NaN,NaN,2016-01-25 04:49:38 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",When bae says they can't go out but you see them with someone else that same night. 5/10 &amp; 10/10 for heartbroken pup https://t.co/aenk0KpoWM,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/691483041324204033/photo/1,https://twitter.com/dog_rates/status/691483041324204033/photo/1,https://twitter.com/dog_rates/status/691483041324204033/photo/1,https://twitter.com/dog_rates/status/691483041324204033/photo/1",5,10,None,None,None,None,None
1583,687102708889812993,NaN,NaN,2016-01-13 02:43:46 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Army of water dogs here. None of them know where they're going. Have no real purpose. Aggressive barks. 5/10 for all https://t.co/A88x73TwMN,NaN,NaN,NaN,https://twitter.com/dog_rates/status/687102708889812993/photo/1,5,10,None,None,None,None,None
1618,684969860808454144,6.849598e+17,4.196984e+09,2016-01-07 05:28:35 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","For those who claim this is a goat, u are wrong. It is not the Greatest Of All Time. The rating of 5/10 should have made that clear. Thank u",NaN,NaN,NaN,NaN,5,10,None,None,None,None,None
1619,684959798585110529,NaN,NaN,2016-01-07 04:48:36 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Jerry. He's a neat dog. No legs (tragic). Has more horns than a dog usually does. Bark is unique af. 5/10 https://t.co/85q7xlplsJ,NaN,NaN,NaN,https://twitter.com/dog_rates/status/684959798585110529/photo/1,5,10,Jerry,None,None,None,None
1624,684880619965411328,NaN,NaN,2016-01-06 23:33:58 +0000,"<a href=""http://twitter.com/downlo

In [24]:
archive.timestamp.sort_values()

2355    2015-11-15 22:32:08 +0000
2354    2015-11-15 23:05:30 +0000
2353    2015-11-15 23:21:54 +0000
2352    2015-11-16 00:04:52 +0000
2351    2015-11-16 00:24:50 +0000
2350    2015-11-16 00:30:50 +0000
2349    2015-11-16 00:35:11 +0000
2348    2015-11-16 00:49:46 +0000
2347    2015-11-16 00:55:59 +0000
2346    2015-11-16 01:01:59 +0000
2345    2015-11-16 01:22:45 +0000
2344    2015-11-16 01:52:02 +0000
2343    2015-11-16 01:59:36 +0000
2342    2015-11-16 02:38:37 +0000
2341    2015-11-16 03:22:39 +0000
2340    2015-11-16 03:44:34 +0000
2339    2015-11-16 03:55:04 +0000
2338    2015-11-16 04:02:55 +0000
2337    2015-11-16 14:57:41 +0000
2336    2015-11-16 15:14:19 +0000
2335    2015-11-16 16:11:11 +0000
2334    2015-11-16 16:37:02 +0000
2333    2015-11-16 19:31:45 +0000
2332    2015-11-16 20:01:42 +0000
2331    2015-11-16 20:32:58 +0000
2330    2015-11-16 21:10:36 +0000
2329    2015-11-16 21:54:18 +0000
2328    2015-11-16 23:23:41 +0000
2327    2015-11-17 00:06:54 +0000
2326    2015-1

In [25]:
archive[archive.duplicated() ==True]
archive[archive.tweet_id.duplicated() ==True]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [26]:
archive.source.value_counts()

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                        91  
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                     33  
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>    11  
Name: source, dtype: int64

In [27]:
sum(archive.expanded_urls.isnull())

59

In [28]:
sum(archive.in_reply_to_user_id.isnull())

2278

In [29]:
sum(archive.retweeted_status_user_id.isnull())

2175

#### 2.2  imagePrediction table

In [30]:
imagePrediction.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [31]:
imagePrediction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [32]:
imagePrediction.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


In [33]:
imagePrediction['p1_conf'].describe().apply(lambda x: format(x, 'f'))

count    2075.000000
mean     0.594548   
std      0.271174   
min      0.044333   
25%      0.364412   
50%      0.588230   
75%      0.843855   
max      1.000000   
Name: p1_conf, dtype: object

In [34]:
imagePrediction['p2_conf'].describe().apply(lambda x: format(x, 'f'))

count    2075.000000
mean     0.134589   
std      0.100666   
min      0.000000   
25%      0.053886   
50%      0.118181   
75%      0.195566   
max      0.488014   
Name: p2_conf, dtype: object

In [35]:
imagePrediction['p3_conf'].describe().apply(lambda x: format(x, 'f'))

count    2075.000000
mean     0.060324   
std      0.050906   
min      0.000000   
25%      0.016222   
50%      0.049444   
75%      0.091808   
max      0.273419   
Name: p3_conf, dtype: object

In [36]:
imagePrediction.p1.value_counts()

golden_retriever             150
Labrador_retriever           100
Pembroke                     89 
Chihuahua                    83 
pug                          57 
chow                         44 
Samoyed                      43 
toy_poodle                   39 
Pomeranian                   38 
malamute                     30 
cocker_spaniel               30 
French_bulldog               26 
miniature_pinscher           23 
Chesapeake_Bay_retriever     23 
seat_belt                    22 
Siberian_husky               20 
German_shepherd              20 
Staffordshire_bullterrier    20 
Cardigan                     19 
web_site                     19 
beagle                       18 
Eskimo_dog                   18 
Shetland_sheepdog            18 
Maltese_dog                  18 
teddy                        18 
Lakeland_terrier             17 
Rottweiler                   17 
Shih-Tzu                     17 
kuvasz                       16 
Italian_greyhound            16 
          

In [37]:
imagePrediction.p2.value_counts()

Labrador_retriever                104
golden_retriever                  92 
Cardigan                          73 
Chihuahua                         44 
Pomeranian                        42 
Chesapeake_Bay_retriever          41 
French_bulldog                    41 
toy_poodle                        37 
cocker_spaniel                    34 
Siberian_husky                    33 
miniature_poodle                  33 
beagle                            28 
Pembroke                          27 
Eskimo_dog                        27 
collie                            27 
kuvasz                            26 
Italian_greyhound                 22 
Pekinese                          21 
American_Staffordshire_terrier    21 
chow                              20 
miniature_pinscher                20 
toy_terrier                       20 
malinois                          20 
Samoyed                           20 
Norwegian_elkhound                19 
Boston_bull                       19 
Staffordshir

In [38]:
imagePrediction.p3.value_counts()

Labrador_retriever                79
Chihuahua                         58
golden_retriever                  48
Eskimo_dog                        38
kelpie                            35
kuvasz                            34
Staffordshire_bullterrier         32
chow                              32
cocker_spaniel                    31
beagle                            31
toy_poodle                        29
Pekinese                          29
Pomeranian                        29
Pembroke                          27
Chesapeake_Bay_retriever          27
Great_Pyrenees                    27
malamute                          26
French_bulldog                    26
American_Staffordshire_terrier    24
Cardigan                          23
pug                               23
basenji                           21
bull_mastiff                      20
toy_terrier                       20
Siberian_husky                    19
Boston_bull                       17
Shetland_sheepdog                 17
L

In [39]:
imagePrediction.p1_dog.value_counts()

True     1532
False    543 
Name: p1_dog, dtype: int64

In [40]:
imagePrediction.p2_dog.value_counts()

True     1553
False    522 
Name: p2_dog, dtype: int64

In [41]:
imagePrediction.p3_dog.value_counts()

True     1499
False    576 
Name: p3_dog, dtype: int64

#### 2.3  imagePrediction table

In [42]:
tweet_json.head()

,tweet_id,retweet_count,favorite_count
0,1090301319825436672,2,305
1,1090300780270149632,11,1226
2,1090300624657252352,6989,79691
3,1090060665903927296,1,143
4,1090060504788197377,1,116


In [43]:
tweet_json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2157 entries, 0 to 2156
Data columns (total 3 columns):
tweet_id          2157 non-null int64
retweet_count     2157 non-null int64
favorite_count    2157 non-null int64
dtypes: int64(3)
memory usage: 50.6 KB


In [44]:
tweet_json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2157 entries, 0 to 2156
Data columns (total 3 columns):
tweet_id          2157 non-null int64
retweet_count     2157 non-null int64
favorite_count    2157 non-null int64
dtypes: int64(3)
memory usage: 50.6 KB


In [45]:
tweet_json.describe()

,tweet_id,retweet_count,favorite_count
count,2.157000e+03,2157.000000,2157.000000
mean,1.043777e+18,2811.560037,10408.850255
std,1.998247e+16,19050.964888,40845.247559
min,1.027963e+18,0.000000,0.000000
25%,1.028138e+18,0.000000,0.000000
50%,1.029407e+18,0.000000,1.000000
75%,1.060962e+18,4.000000,357.000000
max,1.090301e+18,466025.000000,690028.000000


#### 2.4 Merge the table names a

In [46]:
all_columns= pd.Series(list(archive) +list(imagePrediction)+ list(tweet_json))
all_columns[all_columns.duplicated()]

17    tweet_id
29    tweet_id
dtype: object

# Clean

### 1. Tidiness

copy all data frames before making any cleaning process using copy() function to save them in case you need to return back with the original

In [47]:
d1 = archive.copy()

In [48]:
Archive_clean = archive.copy()
imagePrediction_clean = imagePrediction.copy()

#### Tidiness
##### doggo, floofer, pupper, and puppo column in archive table are all describing dog age stage.

##### Define
The doggo, floofer, pupper, and puppo column should be merged into one column called dog_stage to reduce redundency. 

##### Code

In [49]:
Archive_clean.replace('None', np.nan, inplace=True)
Archive_clean['dog_stage'] = Archive_clean[['doggo', 'floofer', 'pupper', 'puppo']].fillna('').sum(axis=1).astype(str)
#Drop redundent columns
Archive_clean = Archive_clean.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis=1)

#### Test 

In [50]:
Archive_clean['dog_stage'].value_counts()

                1976
pupper          245 
doggo           83  
puppo           29  
doggopupper     12  
floofer         9   
doggopuppo      1   
doggofloofer    1   
Name: dog_stage, dtype: int64

There are 1 doggo and puppo value and 1 doggo and floofer value, and 12 doggo pupper value.This may happen when text has both word.  I will need to check these rows one by one to verify if it is right. 

In [51]:
Archive_clean[Archive_clean['dog_stage'] == 'doggopuppo'] #It's clearly a puppo

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,dog_stage
191,855851453814013952,NaN,NaN,2017-04-22 18:31:02 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here's a puppo participating in the #ScienceMarch. Cleverly disguising her own doggo agenda. 13/10 would keep the planet habitable for https://t.co/cMhq16isel,NaN,NaN,NaN,https://twitter.com/dog_rates/status/855851453814013952/photo/1,13,10,NaN,doggopuppo


In [52]:
Archive_clean.loc[Archive_clean['dog_stage'] == 'doggopuppo', 'dog_stage'] = 'puppo'

In [53]:
Archive_clean[Archive_clean['dog_stage'] == 'doggofloofer']#this should be a floofer

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,dog_stage
200,854010172552949760,NaN,NaN,2017-04-17 16:34:26 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","At first I thought this was a shy doggo, but it's actually a Rare Canadian Floofer Owl. Amateurs would confuse the two. 11/10 only send dogs https://t.co/TXdT3tmuYk",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/854010172552949760/photo/1,https://twitter.com/dog_rates/status/854010172552949760/photo/1",11,10,NaN,doggofloofer


In [54]:
Archive_clean.loc[Archive_clean['dog_stage'] == 'doggofloofer','dog_stage']  = 'floofer'

In [55]:
Archive_clean[Archive_clean['dog_stage'] == 'doggopupper'] 
#Some tweets have both doggo and pupper for two dogs, this will be kept and used as one categorty

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,dog_stage
460,817777686764523521,NaN,NaN,2017-01-07 16:59:28 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Dido. She's playing the lead role in ""Pupper Stops to Catch Snow Before Resuming Shadow Box with Dried Apple."" 13/10 (IG: didodoggo) https://t.co/m7isZrOBX7",NaN,NaN,NaN,https://twitter.com/dog_rates/status/817777686764523521/video/1,13,10,Dido,doggopupper
531,808106460588765185,NaN,NaN,2016-12-12 00:29:28 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here we have Burke (pupper) and Dexter (doggo). Pupper wants to be exactly like doggo. Both 12/10 would pet at same time https://t.co/ANBpEYHaho,NaN,NaN,NaN,https://twitter.com/dog_rates/status/808106460588765185/photo/1,12,10,NaN,doggopupper
565,802265048156610565,7.331095e+17,4.196984e+09,2016-11-25 21:37:47 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","Like doggo, like pupper version 2. Both 11/10 https://t.co/9IxWAXFqze",NaN,NaN,NaN,https://twitter.com/dog_rates/status/802265048156610565/photo/1,11,10,NaN,doggopupper
575,801115127852503040,NaN,NaN,2016-11-22 17:28:25 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Bones. He's being haunted by another doggo of roughly the same size. 12/10 deep breaths pupper everything's fine https://t.co/55Dqe0SJNj,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/801115127852503040/photo/1,https://twitter.com/dog_rates/status/801115127852503040/photo/1",12,10,Bones,doggopupper
705,785639753186217984,NaN,NaN,2016-10-11 00:34:48 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Pinot. He's a sophisticated doggo. You can tell by the hat. Also pointier than your average pupper. Still 10/10 would pet cautiously https://t.co/f2wmLZTPHd,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/785639753186217984/photo/1,https://twitter.com/dog_rates/status/785639753186217984/photo/1",10,10,Pinot,doggopupper
733,781308096455073793,NaN,NaN,2016-09-29 01:42:20 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine - Make a Scene</a>","Pupper butt 1, Doggo 0. Both 12/10 https://t.co/WQvcPEpH2u",NaN,NaN,NaN,https://vine.co/v/5rgu2Law2ut,12,10,NaN,doggopupper
778,775898661951791106,NaN,NaN,2016-09-14 03:27:11 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","RT @dog_rates: Like father (doggo), like son (pupper). Both 12/10 https://t.co/pG2inLaOda",7.331095e+17,4.196984e+09,2016-05-19 01:38:16 +0000,"https://twitter.com/dog_rates/status/733109485275860992/photo/1,https://twitter.com/dog_rates/status/733109485275860992/photo/1",12,10,NaN,doggopupper
822,770093767776997377,NaN,NaN,2016-08-29 03:00:36 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",RT @dog_rates: This is just downright precious af. 12/10 for both pupper and doggo https://t.co/o5J479bZUC,7.410673e+17,4.196984e+09,2016-06-10 00:39:48 +0000,"https://twitter.com/dog_rates/status/741067306818797568/photo/1,https://twitter.com/dog_rates/status/741067306818797568/photo/1",12,10,just,doggopupper
889,759793422261743616,NaN,NaN,2016-07-31 16:50:42 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","Meet Maggie &amp; Lila. Maggie is the doggo, Lila is the pupper. They are sisters. Both 12/10 would pet at the same time https://t.co/MYwR4DQKll",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/759793422261743616/photo/1,https://twitter.com/dog_rates/status/759793422261743616/photo/1",12,10,Maggie,doggopupper
956,751583847268179968,NaN,NaN,2016-07-09 01:08:47 +0000,"<a href=""http://twitter.com/download/iphone"" r

In [56]:
Archive_clean['dog_stage'].value_counts()

               1976
pupper         245 
doggo          83  
puppo          30  
doggopupper    12  
floofer        10  
Name: dog_stage, dtype: int64

The result looks correct. 
Let's work on the second tidy data issue - The column names in imagePrediction are not specific and not easy to undstand.
### Define 

Rename column names -(p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog) in imagePrediction to make them user friendly

### Code

In [57]:
imagePrediction_clean.rename(index=str, columns={"p1" :"prediction1", "p1_conf":"prediction_confidence1","p1_dog" : "dogbreed_indicator1",
                                                "p2" :"prediction2", "p2_conf":"prediction_confidence2","p2_dog" : "dogbreed_indicator2",
                                                 "p3" :"prediction3", "p3_conf":"prediction_confidence3","p3_dog" : "dogbreed_indicator3",
                                                 "img_num" : "most_confident_img_num"}, inplace =True)

### Test

In [58]:
imagePrediction_clean.head(2)

,tweet_id,jpg_url,most_confident_img_num,prediction1,prediction_confidence1,dogbreed_indicator1,prediction2,prediction_confidence2,dogbreed_indicator2,prediction3,prediction_confidence3,dogbreed_indicator3
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True


The columns name in imagePrediction table is changed and look make sense.

### Define 

The last step should be merging tweets_json table with Twitter_Archive Table on tweet_id column.

### Code

In [59]:
Archive_clean = pd.merge(Archive_clean, tweet_json,
                            on=['tweet_id'], how='left')

Here, I will keep Archive_clean table and image prediction table both for tidiness of data. 

### Test

In [60]:
Archive_clean.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,dog_stage,retweet_count,favorite_count
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,,NaN,NaN
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,,NaN,NaN
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,,NaN,NaN
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13,10,Darla,,NaN,NaN
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",12,10,Franklin,,NaN,NaN


Here, I will keep Archive_clean table and image prediction table both for tidiness of data. 

# Quality issues
Missing data is part of data quality issue.

### Define 
expanded_urls column has 59 url with missing value, I can fill them up by combining 'https://twitter.com/dog_rates/status/' with their tweet id  and connect to '/photo/1'


In [61]:
Archive_clean['expanded_urls'] = Archive_clean['expanded_urls'].fillna(
    value ='https://twitter.com/dog_rates/status/'+ Archive_clean['tweet_id'].astype(str) + '/photo/1')

### Test 

In [67]:
sum(Archive_clean['expanded_urls'].isnull())

0

From the result, we can see all the missing value in expanded_urls column is filled with correct url

in_reply_to_status_id,in_reply_to_user_id all only have 78 non-null value, 2278 missing value, 97% field in these columns are missing. retweeted_status_id,retweeted_status_user_id and retweeted_status_timestamp have 2175 missing value out of 2356, about 92% fields are missing.

# Define

Since we don't use reweet data, I will delete the rows populated with values under these columns and remove these columns which have a large amount of missing value.

# Code  

In [63]:
Archive_clean[Archive_clean["in_reply_to_status_id"].isnull() ==False] 

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,dog_stage,retweet_count,favorite_count
30,886267009285017600,8.862664e+17,2.281182e+09,2017-07-15 16:51:35 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",@NonWhiteHat @MayhewMayhem omg hello tanner you are a scary good boy 12/10 would pet with extreme caution,NaN,NaN,NaN,https://twitter.com/dog_rates/status/886267009285017600/photo/1,12,10,NaN,,NaN,NaN
55,881633300179243008,8.816070e+17,4.738443e+07,2017-07-02 21:58:53 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",@roushfenway These are good dogs but 17/10 is an emotional impulse rating. More like 13/10s,NaN,NaN,NaN,https://twitter.com/dog_rates/status/881633300179243008/photo/1,17,10,NaN,,NaN,NaN
64,879674319642796034,8.795538e+17,3.105441e+09,2017-06-27 12:14:36 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",@RealKentMurphy 14/10 confirmed,NaN,NaN,NaN,https://twitter.com/dog_rates/status/879674319642796034/photo/1,14,10,NaN,,NaN,NaN
113,870726314365509632,8.707262e+17,1.648776e+07,2017-06-02 19:38:25 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",@ComplicitOwl @ShopWeRateDogs &gt;10/10 is reserved for dogs,NaN,NaN,NaN,https://twitter.com/dog_rates/status/870726314365509632/photo/1,10,10,NaN,,NaN,NaN
148,863427515083354112,8.634256e+17,7.759620e+07,2017-05-13 16:15:35 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","@Jack_Septic_Eye I'd need a few more pics to polish a full analysis, but based on the good boy content above I'm leaning towards 12/10",NaN,NaN,NaN,https://twitter.com/dog_rates/status/863427515083354112/photo/1,12,10,NaN,,NaN,NaN
149,863079547188785154,6.671522e+17,4.196984e+09,2017-05-12 17:12:53 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","Ladies and gentlemen... I found Pipsy. He may have changed his name to Pablo, but he never changed his love for the sea. Pupgraded to 14/10 https://t.co/lVU5GyNFen",NaN,NaN,NaN,https://twitter.com/dog_rates/status/863079547188785154/photo/1,14,10,NaN,,NaN,NaN
179,857214891891077121,8.571567e+17,1.806710e+08,2017-04-26 12:48:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",@Marc_IRL pixelated af 12/10,NaN,NaN,NaN,https://twitter.com/dog_rates/status/857214891891077121/photo/1,12,10,NaN,,NaN,NaN
184,856526610513747968,8.558181e+17,4.196984e+09,2017-04-24 15:13:52 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","THIS IS CHARLIE, MARK. HE DID JUST WANT TO SAY HI AFTER ALL. PUPGRADED TO A 14/10. WOULD BE AN HONOR TO FLY WITH https://t.co/p1hBHCmWnA",NaN,NaN,NaN,https://twitter.com/dog_rates/status/856526610513747968/photo/1,14,10,NaN,,NaN,NaN
186,856288084350160898,8.562860e+17,2.792810e+08,2017-04-23 23:26:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",@xianmcguire @Jenna_Marbles Kardashians wouldn't be famous if as a society we didn't place enormous value on what they do. The dogs are very deserving of their 14/10,NaN,NaN,NaN,https://twitter.com/dog_rates/status/856288084350160898/photo/1,14,10,NaN,,NaN,NaN
188,855862651834028034,8.558616e+17,1.943518e+08,2017-04-22 19:15:32 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",@dhmontgomery We also gave snoop dogg a 420/10 but I think that predated your research,NaN,NaN,NaN,https://twitter.com/dog_rates/status/855862651834028034/photo/1,420,10,NaN,,NaN,NaN


In [64]:
Archive_clean = Archive_clean[Archive_clean["in_reply_to_status_id"].isnull()] 
Archive_clean = Archive_clean[Archive_clean["retweeted_status_id"].isnull()] 

In [65]:
Archive_clean.drop(['in_reply_to_status_id','in_reply_to_user_id','retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp'], axis=1, inplace =True)

#### Test 

In [66]:
Archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 11 columns):
tweet_id              2097 non-null int64
timestamp             2097 non-null object
source                2097 non-null object
text                  2097 non-null object
expanded_urls         2097 non-null object
rating_numerator      2097 non-null int64
rating_denominator    2097 non-null int64
name                  1494 non-null object
dog_stage             2097 non-null object
retweet_count         0 non-null float64
favorite_count        0 non-null float64
dtypes: float64(2), int64(3), object(6)
memory usage: 196.6+ KB


Now the table only has 2097 rows and in_reply_to_status_id,in_reply_to_user_id,
retweeted_status_id,retweeted_status_user_id, retweeted_status_timestamp are removed. 

tweet_id column should be of object type, not integer.
 
### Define
tweet_id columns in two will be both changed into  object type. 

In [68]:
Archive_clean['tweet_id'] = Archive_clean['tweet_id'].astype(str)
imagePrediction_clean['tweet_id'] = imagePrediction_clean['tweet_id'].astype(str)

In [69]:
Archive_clean['tweet_id'].dtypes

dtype('O')

The type of tweet id is changed successfully. 

#### Next step will be source columns. There are 4 different kinds of source -- 'Twitter for iPhone','Vine - Make a Scene','Twitter Web Client',and 'TweetDeck'.

### Define 

Source column should be converted into 4 category of source -'Twitter for iPhone','Vine - Make a Scene','Twitter Web Client',and 'TweetDeck'.

In [70]:
sourcemap = {'<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>': 'Twitter for iPhone',
'<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>' : 'Vine - Make a Scene',
'<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>' :'Twitter Web Client',
'<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>':'TweetDeck'}

# Function to apply
def convert_source(Archive_clean):
    if Archive_clean['source'] in sourcemap.keys():
        sourceab = sourcemap[Archive_clean['source']]
        return sourceab
    else:
        return Archive_clean['source']
    
Archive_clean['source'] = Archive_clean.apply(convert_source, axis=1)

#### Test

In [71]:
Archive_clean['source'].value_counts()

Twitter for iPhone     1964
Vine - Make a Scene    91  
Twitter Web Client     31  
TweetDeck              11  
Name: source, dtype: int64

The source column is converted successfully,but the type of column is int64, which should be changed into category.

#### Erroneous datatypes (timestamp, source columns) Erroneous format in value of prediction1-3 dog breed , prediction_confidence1-3  have 6 decimals and Name is extracted from the word after 'This is', a lot of text does not follow this rule,some texts just have 'a', 'an', 'the'. 
### Define
    timestamp should be removed '+0000', year and month variable are created and convert into datetime . 
    p1,p2,p3 dog breed should be all lower case and p1_conf, p2_conf, p3_conf  should be trimmed into 4 decimal.
    Name is very hard to clean, I will replace -'None', 'a', 'an', 'the' with nan value.

### Code 

In [72]:
# To category
Archive_clean['source'] = Archive_clean['source'].astype('category')

# Strip '+0000'
Archive_clean.timestamp  = Archive_clean.timestamp.str.strip('+0000')

# Extract year and month variable
Archive_clean['month']= Archive_clean['timestamp'].str.split('-').str.get(0) + Archive_clean['timestamp'].str.split('-').str.get(1)  
Archive_clean['month'] = Archive_clean['month'].astype(str)

Archive_clean['year']= Archive_clean['timestamp'].str.split('-').str.get(0)

# To datetime
Archive_clean.timestamp = pd.to_datetime(Archive_clean.timestamp)

# To lower case 
imagePrediction_clean['prediction1']= imagePrediction_clean['prediction1'].str.lower()
imagePrediction_clean['prediction2']= imagePrediction_clean['prediction2'].str.lower()
imagePrediction_clean['prediction3']= imagePrediction_clean['prediction3'].str.lower()

# to 4 decimal 
imagePrediction_clean.prediction_confidence1 = imagePrediction_clean.prediction_confidence1.round(4)
imagePrediction_clean.prediction_confidence2 = imagePrediction_clean.prediction_confidence2.round(4)
imagePrediction_clean.prediction_confidence3 = imagePrediction_clean.prediction_confidence3.round(4)

# replace 'None',a', 'an','the' with nan
Archive_clean.name = Archive_clean.name.replace(['a','an','None','the'], np.nan)

#To string
Archive_clean['name'] = Archive_clean['name'].apply(str)

### Test

In [73]:
Archive_clean.timestamp.head()

0   2017-08-01 16:23:56
1   2017-08-01 00:17:27
2   2017-07-31 00:18:03
3   2017-07-30 15:58:51
4   2017-07-29 16:00:24
Name: timestamp, dtype: datetime64[ns]

In [74]:
imagePrediction_clean[['prediction1','prediction2','prediction3']].head()

,prediction1,prediction2,prediction3
0,welsh_springer_spaniel,collie,shetland_sheepdog
1,redbone,miniature_pinscher,rhodesian_ridgeback
2,german_shepherd,malinois,bloodhound
3,rhodesian_ridgeback,redbone,miniature_pinscher
4,miniature_pinscher,rottweiler,doberman


In [75]:
imagePrediction_clean[['prediction_confidence1','prediction_confidence2','prediction_confidence3']].head()

,prediction_confidence1,prediction_confidence2,prediction_confidence3
0,0.4651,0.1567,0.0614
1,0.5068,0.0742,0.0720
2,0.5965,0.1386,0.1162
3,0.4081,0.3607,0.2228
4,0.5603,0.2437,0.1546


In [76]:
Archive_clean.name.value_counts()

nan           672
Charlie       11 
Lucy          11 
Oliver        10 
Cooper        10 
Penny         9  
Tucker        9  
Lola          8  
Sadie         8  
Winston       8  
Toby          7  
Daisy         7  
Stanley       6  
Bella         6  
Jax           6  
Bailey        6  
Bo            6  
Koda          6  
Oscar         6  
Milo          5  
Leo           5  
Buddy         5  
Scout         5  
Rusty         5  
Louis         5  
Dave          5  
Chester       5  
Bentley       5  
Phil          4  
Boomer        4  
             ..  
Eve           1  
Willie        1  
Brian         1  
Carter        1  
Sephie        1  
Chef          1  
Grizzwald     1  
Perry         1  
Buckley       1  
Patch         1  
Tom           1  
Brudge        1  
Donny         1  
Tanner        1  
Tater         1  
Grizz         1  
Kendall       1  
Molly         1  
incredibly    1  
Grey          1  
Butter        1  
Hanz          1  
Keet          1  
Corey         1  
Snicku    

In [77]:
text1 = Archive_clean[Archive_clean['rating_numerator'] == 5]['text']
for x in text1:
    print(re.findall("\d+\.\d+", x))

['13.5']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [78]:
Archive_clean.loc[Archive_clean['rating_denominator'] == 0,['rating_numerator','rating_denominator']] = 13,10
Archive_clean.loc[Archive_clean['rating_denominator'] == 2,['rating_numerator','rating_denominator']] = 9,10
Archive_clean.loc[Archive_clean['tweet_id']== '883482846933004288','rating_numerator'] = 13.5
Archive_clean.loc[Archive_clean['tweet_id'] == 775096608509886464 ,
                  ['rating_numerator','rating_denominator']] = 14,10
Archive_clean.loc[Archive_clean['tweet_id'] == 740373189193256964,
                  ['rating_numerator','rating_denominator']] = 14,10
Archive_clean.loc[Archive_clean['tweet_id'] == 682962037429899265, 
                  ['rating_numerator','rating_denominator']] = 10,10

In [79]:
imagePrediction_clean.loc[imagePrediction_clean['prediction1'].isin(['newfoundland', 'laptop','sea_lion','lhasa','mitten','feather_boa']),'dogbreed_indicator1'] =False
imagePrediction_clean.loc[imagePrediction_clean['prediction2'].isin(['newfoundland', 'laptop','sea_lion','lhasa','mitten','feather_boa']),'dogbreed_indicator2'] =False
imagePrediction_clean.loc[imagePrediction_clean['prediction3'].isin(['newfoundland', 'laptop','sea_lion','lhasa','mitten','feather_boa']),'dogbreed_indicator3'] =False

#### Test

In [80]:
Archive_clean[Archive_clean['tweet_id']== '883482846933004288'][['text','rating_numerator','rating_denominator']]

,text,rating_numerator,rating_denominator
45,"This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948",13.5,10


In [81]:
Archive_clean['rating_denominator'].value_counts()

10     2081
50     3   
11     2   
80     2   
7      1   
170    1   
150    1   
120    1   
110    1   
90     1   
70     1   
40     1   
20     1   
Name: rating_denominator, dtype: int64

In [82]:
imagePrediction_clean[imagePrediction_clean['prediction3'] == 'mitten']

,tweet_id,jpg_url,most_confident_img_num,prediction1,prediction_confidence1,dogbreed_indicator1,prediction2,prediction_confidence2,dogbreed_indicator2,prediction3,prediction_confidence3,dogbreed_indicator3
950,704859558691414016,https://pbs.twimg.com/media/CcgqBNVW8AE76lv.jpg,1,pug,0.2844,True,teddy,0.1563,False,mitten,0.1389,False


In [83]:
imagePrediction_clean[imagePrediction_clean['prediction2'] == 'newfoundland']['dogbreed_indicator2']

16      False
22      False
732     False
1012    False
1846    False
1859    False
1890    False
Name: dogbreed_indicator2, dtype: bool

In [84]:
imagePrediction_clean[imagePrediction_clean['prediction3'] == 'newfoundland']['dogbreed_indicator3']

114     False
491     False
1147    False
1540    False
1842    False
1868    False
1875    False
1994    False
2032    False
Name: dogbreed_indicator3, dtype: bool

In [85]:
Archive_clean['rating'] =Archive_clean['rating_numerator']/Archive_clean['rating_denominator']

### Test

In [86]:
Archive_clean[['rating','rating_numerator','rating_denominator']].head()

,rating,rating_numerator,rating_denominator
0,1.3,13.0,10
1,1.3,13.0,10
2,1.2,12.0,10
3,1.3,13.0,10
4,1.2,12.0,10


Archive_clean['rating'] column is successfully created.

#### Store Files

In [87]:
Archive_clean.to_csv('./archive_master.csv', index=False)
imagePrediction_clean.to_csv('./imagePrediction_master.csv', index=False)

In [88]:
twitter_archive_master = pd.merge(Archive_clean, imagePrediction_clean, on='tweet_id')

In [89]:
Archive_clean.to_csv('./master.csv', index=False)
